In [1]:
import numpy as np

In [2]:
import mnist_reader
X_train, y_train = mnist_reader.load_mnist('data/fashion', kind='train')
X_test, y_test = mnist_reader.load_mnist('data/fashion', kind='t10k')

In [3]:
def inflate_matrixes(flat_thetas, shapes):
    layers  = len(shapes)+1
    sizes = [shape[0] * shape[1] for shape in shapes]
    steps = np.zeros(layers, dtype=int)

    for i in range(layers-1):
        steps[i+1] = steps[i] + sizes[i]
    return[
        flat_thetas[steps[i]: steps[i+1]].reshape(*shapes[i])
        for i in range(layers-1)
    ]


In [4]:
def feed_forward(thetas, X): # Es sera de (784, 60000)
    a = [X]
    for i in range(len(thetas)):
        a.append(
            sigmoid(
                np.matmul( 
                    thetas[i], #Theta tendra (125,785) la primera vez 
                    np.vstack((#Este Vstack tiene (785, 60000)
                        np.ones(len(X_train[0])).reshape(1,len(X_train[0])),
                        a[i]
                    )) 
                )#La mutiplicacion deja (125,60000)
            )#LA sigmoide lo deja de (125,60000)
        )
    return a

In [5]:
sigmoid = lambda x: 1.0/(1.0 + np.exp(-x))

In [6]:
X_train = X_train.T
a = [X_train]

In [10]:
theta0 = np.random.rand(125,785)
theta1 = np.random.rand(10,126)

In [11]:
thetas = [theta0, theta1]

In [18]:
a = feed_forward(thetas, X_train)

In [22]:
a[0].shape

(784, 60000)

In [21]:
a[1].shape

(125, 60000)

In [20]:
a[2].shape

(10, 60000)

# PRUEBAS

In [35]:
shapes = [a.shape, b.shape]

m = a.flatten()
n = b.flatten()

c = np.array([*m, *n])

In [34]:
a =  np.arange(6).reshape(2, 3)
b =  np.arange(6,9).reshape(1, 3)

In [37]:
d = inflate_matrixes(c,shapes)

[array([[0, 1, 2],
       [3, 4, 5]]), array([[6, 7, 8]])]
